# Import packages and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
in_path = "/content/drive/MyDrive/Spring 2023/Data Visualization/Project/college_rankings_merged_with_ScoreCard_data.xlsx"
df = pd.read_excel(in_path, sheet_name = "Sheet1")

# Base columns

In [3]:
meta_cols = ['year','INSTNM'] 
world_cols = ['world_rank_THE','world_rank_RUR','world_rank_CWUR','score_THE','score_RUR','score_CWUR']

# Feature Engineering

In [4]:
UGDS_cols = [
    'UGDS_WHITE',
    'UGDS_BLACK',
    'UGDS_HISP',
    'UGDS_ASIAN',
    'UGDS_AIAN',
    'UGDS_NHPI',
    'UGDS_2MOR',
    'UGDS_NRA',
    'UGDS_UNKN',
]
df['UGDS_diversity'] = -1 * df[UGDS_cols].var(axis=1)

In [5]:
feature_cols = [
    'international_students_THE',
    'UGDS_diversity',
]

df = df[meta_cols+world_cols+feature_cols]
df['coltype'] = "raw"

In [6]:
df

,year,INSTNM,world_rank_THE,world_rank_RUR,world_rank_CWUR,score_THE,score_RUR,score_CWUR,international_students_THE,UGDS_diversity,coltype
0,2012,Harvard University,2,2.0,1.0,93.9,98.500,100.00,0.25,-0.019831,raw
1,2012,Stanford University,2,3.0,3.0,93.9,96.610,89.50,0.22,-0.013371,raw
2,2012,Massachusetts Institute of Technology,7,4.0,2.0,92.3,95.174,91.67,0.33,-0.014674,raw
3,2012,University of Chicago,9,6.0,11.0,90.2,94.456,73.82,0.21,-0.017655,raw
4,2012,University of California-Berkeley,10,49.0,10.0,89.8,82.240,78.55,0.15,-0.018110,raw
...,...,...,...,...,...,...,...,...,...,...,...
678,2015,University of North Carolina at Greensboro,NaN,NaN,897.0,NaN,NaN,44.13,NaN,-0.033599,raw
679,2015,University of Southern Mississippi,NaN,NaN,901.0,NaN,NaN,44.13,NaN,-0.045082,raw
680,2015,University of North Dakota,NaN,NaN,913.0,NaN,NaN,44.12,NaN,-0.069889,raw
681,2015,The University of Texas at El Paso,NaN,NaN,929.0,NaN,NaN,44.10,NaN,-0.073334,raw


In [7]:
# df['female_male_ratio_THE'] = df['female_male_ratio_THE'].astype("string")
# df['female_proportion'] = df[~df['female_male_ratio_THE'].isna()]['female_male_ratio_THE']

# Create min-max scaled columns

In [8]:
df_copy = df.copy()
df_copy[feature_cols] = (df_copy[feature_cols] - df_copy[feature_cols].min())/(df_copy[feature_cols].max() - df_copy[feature_cols].min())


In [9]:
df_copy['coltype'] = 'scaled'
df = pd.concat([df,df_copy]).reset_index(drop=True)

# Create weight columns

In [10]:
df_copy = df[df['coltype'] == 'scaled'].copy()
df_copy[feature_cols] = df_copy[feature_cols].var()/df_copy[feature_cols].var().sum()
# df[weight_cols] = 1/len(weight_cols)

In [11]:
df_copy['coltype'] = 'weight'
df = pd.concat([df,df_copy]).reset_index(drop=True)

# Create score columns

In [12]:
df_copy = df[df['coltype'] == 'raw'].copy()
for col in feature_cols:
  df_copy[col] = df[df['coltype'] == 'scaled'][col].reset_index(drop=True) * \
  df[df['coltype'] == 'weight'][col].reset_index(drop=True)

In [13]:
df_copy['coltype'] = 'score'
df = pd.concat([df,df_copy]).reset_index(drop=True)

# Create diversity columns

In [14]:
df_copy = df[df['coltype'] == 'score'][meta_cols+feature_cols].copy()
df_copy['diversity_index'] = df_copy[feature_cols].sum(axis=1)
df_copy['diversity_index'] = (df_copy['diversity_index'] - df_copy['diversity_index'].min())/\
(df_copy['diversity_index'].max() - df_copy['diversity_index'].min())
df_copy['diversity_rank'] = df_copy['diversity_index'].rank(ascending=False)
df_copy['diversity_year_rank'] = df_copy.groupby('year')['diversity_index'].rank(ascending=False)

In [15]:
df_copy = df_copy[meta_cols+['diversity_index','diversity_rank','diversity_year_rank']]
df = df.merge(df_copy,on=['year','INSTNM'])

In [16]:
df['diversity_index'] = df.mask(df['coltype'] != 'score')['diversity_index']

In [17]:
df = df.sort_values(by=['year','diversity_year_rank'])

# Export

In [19]:
out_path = '/content/drive/MyDrive/Spring 2023/Data Visualization/Project/diversity_df_v2.csv'
df.to_csv(out_path,index=0)

In [18]:
df

,year,INSTNM,world_rank_THE,world_rank_RUR,world_rank_CWUR,score_THE,score_RUR,score_CWUR,international_students_THE,UGDS_diversity,coltype,diversity_index,diversity_rank,diversity_year_rank
8,2012,Massachusetts Institute of Technology,7,4.0,2.0,92.3,95.174,91.67,0.330000,-0.014674,raw,NaN,4.0,1.0
9,2012,Massachusetts Institute of Technology,7,4.0,2.0,92.3,95.174,91.67,0.941176,0.965848,scaled,NaN,4.0,1.0
10,2012,Massachusetts Institute of Technology,7,4.0,2.0,92.3,95.174,91.67,0.432603,0.567397,weight,NaN,4.0,1.0
11,2012,Massachusetts Institute of Technology,7,4.0,2.0,92.3,95.174,91.67,0.407156,0.548020,score,0.979116,4.0,1.0
48,2012,Carnegie Mellon University,21,33.0,43.0,78.4,84.669,51.60,0.350000,-0.018775,raw,NaN,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2015,Yeshiva University,186,81.0,171.0,46.7,80.001,48.11,0.076342,0.000000,score,0.050820,681.0,207.0
2640,2015,Howard University,NaN,NaN,719.0,NaN,NaN,44.33,NaN,-0.089351,raw,NaN,682.0,208.0
2641,2015,Howard University,NaN,NaN,719.0,NaN,NaN,44.33,NaN,0.069674,scaled,NaN,682.0,208.0
2642,2015,Howard University,NaN,NaN,719.0,NaN,NaN,44.33,0.432603,0.567397,weight,NaN,682.0,208.0
